In [1]:
import pyvista as pv 
import numpy as np
from scipy.spatial import distance
import timeit
pv.set_jupyter_backend('trame')
import math

In [7]:
def radify(a):
    angle = a/360*2*np.pi
    return angle
def Intersection_Point(cone1,cone2,cone3):
    
    intersection, C1_split, C2_split = cone1.intersection(cone2)
    intersection2, C3_split, C1_split2 = cone3.intersection(cone1)

    cells1 = intersection.cell_centers()
    cells2 = intersection2.cell_centers()


    points1 = cells1.points
    points2 = cells2.points
    
    empthy_dict = {}
    for i in range(len(points1)):
        for j in range(len(points2)):
            dist = distance.euclidean(points1[i],points2[j])
            empthy_dict[dist] = [points1[i],points2[j]]

    shortest = sorted(empthy_dict)[0]
    p1,p2 = empthy_dict[shortest]

    p = (p1+p2)/2
    
    return p

def Visual_SearchE(Angles1, Angles2, Angles3, Dimensions,P1,P2,P3,Real_Source):
    
    T1min,T1,T1max = Angles1
    T2min,T2,T2max = Angles2
    T3min,T3,T3max = Angles3
    
    X,Y,Z = Dimensions

    L = Y

    p11,p12,p13 = P1
    Cen1 = (p11,p12+L/2,p13)

    p21,p22,p23 = P2
    Cen2 = (p21,p22+L/2,p23)

    p31,p32,p33 = P3
    Cen3 = (p31,p32+L/2,p33)

    C1 = pv.Cone(center=Cen1,angle=T1,direction=(0,-1,0),resolution=100,height=L,capping=False)
    C2 = pv.Cone(center=Cen2,angle=T2,direction=(0,-1,0),resolution=100,height=L,capping=False)
    C3 = pv.Cone(center=Cen3,angle=T3,direction=(0,-1,0),resolution=100,height=L,capping=False)
    
    C1min = pv.Cone(center=Cen1,angle=T1min,direction=(0,-1,0),resolution=100,height=L,capping=False)
    C2min = pv.Cone(center=Cen2,angle=T2min,direction=(0,-1,0),resolution=100,height=L,capping=False)
    C3min = pv.Cone(center=Cen3,angle=T3min,direction=(0,-1,0),resolution=100,height=L,capping=False)
    
    C1max = pv.Cone(center=Cen1,angle=T1max,direction=(0,-1,0),resolution=100,height=L,capping=False)
    C2max = pv.Cone(center=Cen2,angle=T2max,direction=(0,-1,0),resolution=100,height=L,capping=False)
    C3max = pv.Cone(center=Cen3,angle=T3max,direction=(0,-1,0),resolution=100,height=L,capping=False)

    
    
    
    
    p1 = Intersection_Point(C1min,C2min,C3min)
    p2 = Intersection_Point(C1max,C2min,C3min)
    p3 = Intersection_Point(C1max,C2max,C3min)
    p4 = Intersection_Point(C1max,C2max,C3max)
    p5 = Intersection_Point(C1min,C2max,C3max)
    p6 = Intersection_Point(C1min,C2max,C3max)
    p7 = Intersection_Point(C1min,C2max,C3min)
    p8 = Intersection_Point(C1max,C2min,C3max)
    p9 = Intersection_Point(C1min,C2max,C3min)
    
    error_points = np.array([p1,p2,p3,p4,p5,p6,p7,p8,p9])
    
    intersection, C1_split, C2_split = C1.intersection(C2)
    intersection2, C3_split, C1_split2 = C3.intersection(C1)

    source = Intersection_Point(C1,C2,C3)
    
    P = pv.Sphere(center = source)
    
    E1 = pv.Sphere(center = p1)
    E2 = pv.Sphere(center = p2)
    E3 = pv.Sphere(center = p3)
    E4 = pv.Sphere(center = p4)
    E5 = pv.Sphere(center = p5)
    E6 = pv.Sphere(center = p6)
    E7 = pv.Sphere(center = p7)
    E8 = pv.Sphere(center = p8)
    E9 = pv.Sphere(center = p9)
    
    sx,sy,sz = source
    distances = np.zeros(len(error_points))
    i = 0
    for position in error_points:
        x,y,z = position
        x = x-sx
        y = y-sy
        z = z-sz
        distances[i] = np.sqrt(x**2+y**2+z**2)
        i = i+1
    
    e_radius = np.mean(distances)
    
    Esphere = pv.Sphere(center = source, radius = e_radius)
    
    S1 = pv.Cylinder(P1,direction=(0,0,1),resolution=100,radius=1.75,height=10)
    S2 = pv.Cylinder(P2,direction=(0,0,1),resolution=100,radius=1.75,height=10)
    S3 = pv.Cylinder(P3,direction=(0,0,1),resolution=100,radius=1.75,height=10)
    
    Rsource = pv.Sphere(center = Real_Source,radius=2)
    
    
    

    plotter = pv.Plotter(window_size=(600, 400),image_scale=5)
    plotter.background_color = 'w'
    plotter.enable_anti_aliasing()
    plotter.add_mesh(C1, color='b', show_edges=False,opacity=0.3)
    plotter.add_mesh(C2, color='r', show_edges=False,opacity=0.3)
    plotter.add_mesh(C3, color='g', show_edges=False,opacity=0.3)
    plotter.add_mesh(P, color='w', show_edges=False)
    plotter.add_mesh(Esphere, color='w', show_edges=True,opacity = 0.1)
    plotter.add_mesh(Rsource, color='b', show_edges=False)
    
    plotter.add_mesh(E1, color='r', show_edges=False)
    plotter.add_mesh(E2, color='r', show_edges=False)
    plotter.add_mesh(E3, color='r', show_edges=False)
    plotter.add_mesh(E4, color='r', show_edges=False)
    plotter.add_mesh(E5, color='r', show_edges=False)
    plotter.add_mesh(E6, color='r', show_edges=False)
    plotter.add_mesh(E7, color='r', show_edges=False)
    plotter.add_mesh(E8, color='r', show_edges=False)
    plotter.add_mesh(E9, color='r', show_edges=False)
    
    plotter.add_mesh(intersection,color='y', line_width=5)
    plotter.add_mesh(intersection2,color='pink', line_width=5)
    plotter.add_mesh(S1,color='grey')
    plotter.add_mesh(S2,color='grey')
    plotter.add_mesh(S3,color='grey')

    
    plotter.show(jupyter_backend='trame',screenshot='Final_Cones.png')

    print('The calculated source position is at',source,"the error radius is",e_radius)

In [8]:
"""
Data 23.02.23
"""
#A1 = AngRange(14.72,13.57)
#A2 = AngRange(15.05,12.25)
A3 = (34.06-5.97,34.06,34.06+5.97)
A1 = (10,20.44,32.3)
A2 = (10,16.19,30.99)

P1=(17.3,0,17.4)
P2=(0,0,10.3)
P3=(32,0,10.5)

Real_Source = (9,34,2)

start = timeit.default_timer()

Visual_SearchE(A1,A2,A3,(34,80,24),P1,P2,P3,Real_Source)

stop = timeit.default_timer()

print('Time taken: ', stop - start)


Widget(value="<iframe src='http://localhost:59962/index.html?ui=P_0x205122c1ac0_1&reconnect=auto' style='width…

The calculated source position is at [ 9.37543201 33.86798096  7.58033252] the error radius is 15.668179078907299
Time taken:  7.959609599999993


In [4]:
(17.9)**3*4/3*np.pi

24024.131824329368

In [5]:
np.sqrt((9-8.834)**2+(34-34.521)**2+(2-7.72071409)**2)

5.746787511255705

In [6]:
math.dist([ 8.83379197,34.52101612 ,7.72071409],[9,34,2])

5.746794985560288